In [23]:
# =================================================================
# VALIDAÇÃO ESTRUTURAL COPSOQ - SIMULAÇÃO E ANÁLISE (PROMAX)
# =================================================================

# Instalação e Importação
!pip install factor_analyzer
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from google.colab import files
import time

# Upload do arquivo de mapeamento
print("--- PASSO 1: Selecione o arquivo 'COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv' ---")
uploaded = files.upload()
nome_map = list(uploaded.keys())[0]
mapping = pd.read_csv(nome_map)

# Configurações de Reprodutibilidade e Simulação
SEED = 42
np.random.seed(SEED)
N_AMOSTRA = 500

# Definindo correlações teóricas entre os fatores
# ISP, IAT, NSP
latent_cov = [
    [1.0, -0.3, -0.4], # ISP correlaciona negativamente com Apoio e Satisfação
    [-0.3, 1.0, 0.5],  # IAT correlaciona positivamente com Satisfação
    [-0.4, 0.5, 1.0]
]

latent_factors = np.random.multivariate_normal([0, 0, 0], latent_cov, N_AMOSTRA)

# 4. Geração da Base de Dados (Escala Likert 1-5)
df_simulada = pd.DataFrame()
for idx, row in mapping.iterrows():
    item_id = f"Item_{row['Nº']}"
    f_idx = 0 if 'ISP' in row['Indicador'] else (1 if 'IAT' in row['Indicador'] else 2)

    # Carga fatorial de 0.7 + ruído
    raw = 0.7 * latent_factors[:, f_idx] + np.random.normal(0, 0.5, N_AMOSTRA)
    scaled = (raw - raw.min()) / (raw.max() - raw.min())
    df_simulada[item_id] = np.round(scaled * 4 + 1).astype(int)

# Análise de Fatores Principais (PAF) com Rotação PROMAX
fa = FactorAnalyzer(n_factors=3, method='principal', rotation='promax')
fa.fit(df_simulada)

# Extração de Resultados
# Cargas Fatoriais (Pattern Matrix)
loadings = pd.DataFrame(fa.loadings_, index=df_simulada.columns, columns=['ISP', 'IAT', 'NSP'])

# Matriz de Correlação entre Fatores (O que o revisor pediu especificamente)
factor_corr = pd.DataFrame(fa.phi_, index=['ISP', 'IAT', 'NSP'], columns=['ISP', 'IAT', 'NSP'])

# Cálculo do Alfa de Cronbach
def cronbach(df, ind):
    cols = [f"Item_{i}" for i in mapping[mapping['Indicador'].str.contains(ind)]['Nº']]
    sub = df[cols]
    k = sub.shape[1]
    return (k / (k - 1)) * (1 - (sub.var(ddof=1).sum() / sub.sum(axis=1).var(ddof=1)))

print("\n" + "="*40)
print("RESULTADOS DA VALIDAÇÃO")
print(f"Alfa ISP: {cronbach(df_simulada, 'ISP'):.3f}")
print(f"Alfa IAT: {cronbach(df_simulada, 'IAT'):.3f}")
print(f"Alfa NSP: {cronbach(df_simulada, 'NSP'):.3f}")
print("="*40)
print("\nMatriz de Correlação entre Fatores (Promax):")
print(factor_corr)

# 7. Geração e Download dos Arquivos
arquivos = {
    'base_simulada_500_respostas.csv': df_simulada,
    'cargas_fatoriais_promax.csv': loadings,
    'matriz_correlacao_fatores.csv': factor_corr
}

print("\n--- PASSO 2: Iniciando downloads automáticos... ---")
for nome, data in arquivos.items():
    data.to_csv(nome)
    files.download(nome)
    time.sleep(1) # Pequena pausa para garantir que o navegador processe os downloads

print("\nVerifique se o seu navegador não bloqueou os pop-ups de download!")

--- PASSO 1: Selecione o arquivo 'COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv' ---


Saving COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv.xlsx - perguntas-dimensões.csv to COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv.xlsx - perguntas-dimensões (2).csv

RESULTADOS DA VALIDAÇÃO
Alfa ISP: 0.963
Alfa IAT: 0.916
Alfa NSP: 0.919

Matriz de Correlação entre Fatores (Promax):
          ISP       IAT       NSP
ISP  1.000000 -0.237689 -0.341046
IAT -0.237689  1.000000  0.406212
NSP -0.341046  0.406212  1.000000

--- PASSO 2: Iniciando downloads automáticos... ---


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Verifique se o seu navegador não bloqueou os pop-ups de download!
